In [9]:
import sqlite3
from pathlib import Path

shows_dir = Path("../data/shows")


def get_podcast_stats_by_name(partial_name):
    conn = sqlite3.connect("../data/podcasts.db")
    cursor = conn.cursor()

    query = """
        SELECT
            p.name,
            p.show_id,
            p.category,
            r.country,
            r.rank,
            s.rank_score
        FROM podcast p
        LEFT JOIN ranking r ON p.id = r.podcast_id
        LEFT JOIN score s ON p.id = s.podcast_id
        WHERE p.name LIKE ?
        ORDER BY r.chart_date DESC, s.date DESC;
    """
    podcast_data = {}
    try:
        search_term = f"%{partial_name}%"
        cursor.execute(query, (search_term,))
        rows = cursor.fetchall()

        if not rows:
            print("No podcast found matching:", partial_name)
            return

        for name, show_id, category, country, rank, rank_score in rows:
            if name not in podcast_data:
                podcast_data[name] = {
                    "show_id": show_id,
                    "category": category,
                    "rankings": {},
                    "latest_score": None,
                }

            if country and rank is not None:
                if country not in podcast_data[name]["rankings"]:
                    podcast_data[name]["rankings"][country] = rank

            if rank_score is not None and podcast_data[name]["latest_score"] is None:
                podcast_data[name]["latest_score"] = rank_score

    finally:
        conn.close()
        return podcast_data


podcast_data = get_podcast_stats_by_name("The Duran")
if podcast_data:
    for name, data in podcast_data.items():
        print(f"\nPodcast: {name}")
        print(f"Show ID: {data['show_id']}")
        print(f"Category: {data['category']}")
        print("Country Rankings:")
        for country, rank in data["rankings"].items():
            print(f"  {country}: #{rank}")
        print(f"Overall Score: {data['latest_score']}")


Podcast: The Duran Podcast
Show ID: 1442883993
Category: news
Country Rankings:
  gb: #158
  nz: #105
  ca: #178
  au: #144
Overall Score: 0.5319
